In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
from pprint import pprint
import pickle

In [2]:
def scrapweb(url,driver):
    driver.get(url)
#     driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source,'lxml')
    listprop = driver.find_elements_by_class_name('strip_list')
    obj_dict={}
    dfs=pd.DataFrame()
    for obj in tqdm(listprop):
#         title=obj.find_elements_by_class_name('strip_list wow fadeIn animated')
        reserve_price=obj.find_element_by_class_name('rating').text.replace('Reserve Price: ','')
        reserve_price=reserve_price.replace(',','')
        auction_id =obj.find_element_by_tag_name('small').text.replace('AUCTION ID:','')
        title=obj.find_element_by_tag_name('h5').text
        desc=obj.find_element_by_tag_name('p').text
        date =obj.find_element_by_class_name('btn_listing').text.replace('Auction Date: ','')
        li=obj.find_element_by_tag_name("ul").find_elements_by_tag_name("li")
        li.pop(0)
        for l in li:
            url=l.find_element_by_tag_name("a").get_attribute("href")
            obj_dict[auction_id] = {
                "Title" : title, 
                 "url":url,
                "Description": desc,
                 "Date":date,
                "Auction_id":auction_id,
                "Reserve_Price":reserve_price 
                }
        df=pd.DataFrame(obj_dict).T
    dfs=dfs.append(df)
    return dfs

In [ ]:
df1 = pd.DataFrame()
driver = webdriver.Safari()
driver.get('https://www.eauctionsindia.com/')
# driver.implicitly_wait(5)
soup = BeautifulSoup(driver.page_source,'lxml')
listobjlinks = driver.find_element_by_id("tab_city").find_elements_by_tag_name("a")
urls_tab1 =[_.get_attribute("href") for _ in listobjlinks]
driver.close()
driver = webdriver.Safari()
for obj in tqdm(urls_tab1):
    df1=df1.append(scrapweb(obj,driver))
# print(df1.head())
driver.close()

  6%|▌         | 1/18 [00:00<00:02,  7.75it/s]

In [ ]:
df1.head()

In [ ]:
new = df1['Date'].str.split(" ", n = 1, expand = True)
df1['Date']=new[0]
df1['Time']=new[1]
print(df1)

In [ ]:
df1.shape

In [ ]:
df2 = pd.DataFrame()
driver = webdriver.Safari()
driver.get('https://www.eauctionsindia.com/')
# driver.implicitly_wait(5)
soup = BeautifulSoup(driver.page_source,'lxml')
listobjlinks = driver.find_element_by_id("tab_bank").find_elements_by_tag_name("a")
urls_tab1 =[_.get_attribute("href") for _ in listobjlinks]
for obj in tqdm(urls_tab1):
    df2=df2.append(scrapweb(obj,driver))
# print(df2.head())
driver.close()

In [ ]:
new = df2['Date'].str.split(" ", n = 1, expand = True)
df2['Date']=new[0]
df2['Time']=new[1]


In [ ]:
df2